<a href="https://colab.research.google.com/github/baydex/acciones/blob/master/Acciones_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalacion de librerias e imports

In [ ]:
!pip install selenium

In [ ]:
!apt-get update
!apt-get install -y chromium-chromedriver

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 229 kB in 2s (127 kB/s)
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
chromium-chromedriver is already the newest version (1:85.0.4183.83-0ubuntu2.22.04.1).
0

In [ ]:
import time
import numpy as np
import copy
import json
import random

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.remote.webelement import WebElement

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from IPython.display import display, HTML, clear_output

import tensorflow as tf
from tensorflow.keras.regularizers import l1, l2

# Codigo de objetos usados

In [ ]:
# @title ScraperAcciones: Objeto que raspa datos dada una accion, un plazo y un alias (opcional)
class ScraperAcciones():
  def __init__(self,accion, plazo, alias=""):
    self.url = None
    self.alias = alias if alias != "" else accion
    self.accion = accion
    self.plazo = plazo
    self.datos = str()
    self.__makeUrl(accion, plazo)
    self.makeOptions()
    self.makeDriver()

  def __makeUrl(self, accion, plazo):
    self.url = f"https://www.google.com/finance/quote/{accion}?window={plazo}"

  def makeOptions(self):
    self.options = webdriver.ChromeOptions()
    self.options.add_argument('--headless')
    self.options.add_argument('--no-sandbox')
    self.options.add_argument('--disable-dev-shm-usage')

  def makeDriver(self):
    self.driver = webdriver.Chrome(self.options)

  def searchData(self):
    self.driver.get(self.url)
    time.sleep(5)
    self.searchGraph()
    self.searchPrice()
    self.searchPerformance()
    self.driver.quit()

  def searchGraph(self):
    paths = self.searchPath()
    self.filterPaths(paths)
    self.findGraficoBruto()
    self.processData()

  def searchPath(self):
    return self.driver.find_elements(by=By.TAG_NAME, value="path")

  def filterPaths(self, paths):
    datos_con_borde = {"d":""}
    datosBrutos = {"d":""}
    for path in paths:
      data = {
              "path": path,
              "d": path.get_attribute("d")
              }
      if(len(data["d"]) > len(datos_con_borde["d"])):
        datosBrutos = datos_con_borde
        datos_con_borde = data
      elif(len(datos_con_borde["d"]) > len(data["d"]) > len(datosBrutos["d"])):
        datosBrutos = data
    self.datosBrutos = datosBrutos["d"]

  def findGraficoBruto(self):
    self.graficoBruto = self.driver.execute_script(
                                          f"""return document.querySelector("path[d='{self.datosBrutos}']").closest('svg');""",
                                          self.datosBrutos)
    self.graficoBruto = self.graficoBruto.get_attribute('outerHTML')


  def processData(self):
    self.eliminarLetras()
    self.datos = list()
    self.guardarCoordenadas()
    self.datos = np.array(self.datos)

  def guardarCoordenadas(self):
    for dato in self.datosSinLetras:
      numericos = list()
      strings = dato.split(",")
      for string in strings[-2:]:
        numericos.append(float(string))
      self.datos.append(numericos)

  def eliminarLetras(self):
    self.datosSinLetras = self.datosBrutos[1:]
    self.datosSinLetras = self.datosSinLetras.split("C")

  def searchPrice(self):
    self.lastPriceContainer = self.driver.find_element(by=By.CSS_SELECTOR, value="[data-last-price]")
    self.price = self.lastPriceContainer.get_attribute("data-last-price")
    self.price = self.price.replace(",", "")
    self.price = float(self.price)

  def searchPerformance(self):
    performanceBruto = self.lastPriceContainer.find_element(by=By.CSS_SELECTOR, value="[aria-label]")
    performanceBruto = performanceBruto.get_attribute("aria-label")
    performanceDividido = performanceBruto.split(" ")
    porcentaje = performanceDividido[-1]
    porcentaje = porcentaje.replace("%", "")
    porcentaje = porcentaje.replace(",", "")
    porcentaje = float(porcentaje)
    porcentaje = porcentaje/100
    if(performanceDividido[0] == "Up"):
      self.performance = 1 + porcentaje
    else:
      self.performance = 1 - porcentaje

  def showGraph(self):
    html_code = f"""
    <h1>
      {self.alias} - {self.plazo}
    </h1>
    <p>
      Precio inicial {self.price / self.performance} - Precio actual: {self.price}
    </p>
    <p>
      Rendimiento {self.performance}
    </p>
    <div class="svg-datos">
    {self.graficoBruto}
    </div>
    <script>
      svgs = document.querySelectorAll(".svg-datos");
          for (let svg of svgs){{
              path = svg.querySelectorAll("path")[1];
              x = path.getBoundingClientRect().width;
              y = path.getBoundingClientRect().height;
              svg.style.width = x*1.2 + "px";
              svg.style.height = y*1.5 + "px";
          }}
    </script>
    """
    display(HTML(html_code))
  def dumpData(self):
    return {
    "url" : self.url,
    "alias" : self.alias,
    "accion" : self.accion,
    "plazo" : self.plazo,
    "datos" : self.datos,
    "datosBrutos" : self.datosBrutos,
    "graficoBruto" : self.graficoBruto,
    "price" : self.price,
    "performance" : self.performance,
    }
  def loadData(self, data):
    self.url = data["url"]
    self.alias = data["alias"]
    self.accion = data["accion"]
    self.plazo = data["plazo"]
    self.datos = np.array(data["datos"])
    self.datosBrutos = data["datosBrutos"]
    self.graficoBruto = data["graficoBruto"]
    self.price = data["price"]
    self.performance = data["performance"]

In [ ]:
# @title ContenedorAcciones: Objeto que contiene, carga y guarda las acciones y su respectivo scraper con sus datos
class ContenedorAcciones:
  def __init__(self):
    self.listaAcciones = None

  class PercentProgressControler:
    def __init__(self, tamañoMuestra):
      self.progreso = 0
      self.incremento = round(100 / tamañoMuestra, 2)

    def progresar(self):
      self.progreso = self.progreso + self.incremento

    def display(self):
      clear_output(wait=True)
      print(f"{self.progreso}%")


  def buscarDatos(self, listaAcciones, plazo = "5Y", show_progress = False):
    controladorProgreso = self.PercentProgressControler(len(listaAcciones))
    self.listaAcciones = list()
    for accion in listaAcciones:
      if show_progress:
        self.showProgress(controladorProgreso)
      self.getScraperData(accion, plazo)
    controladorProgreso.display()

  def showProgress(self, controladorProgreso):
    controladorProgreso.display()
    controladorProgreso.progresar()

  def getScraperData(self, accion, plazo):
      accionScraper = ScraperAcciones(accion["nombre"], plazo , accion["alias"])
      accionScraper.searchData()
      accion["accion"] = accionScraper
      self.listaAcciones.append(accion)

  def getData(self):
    return self.listaAcciones

  def saveData(self, ruta, display=False):
    dataObject = self.saveObjectVariables()
    self.saveFile(ruta, dataObject)
    self.parseDictionaryToScraper()
    if(display):
      print(f"Los datos se han guardado en {ruta}")

  def saveObjectVariables(self):
    serializable = list()

    for indice in range(len(self.listaAcciones)):
      serializable.append(self.listaAcciones[indice])
      serializable[indice]["accion"] = self.listaAcciones[indice]["accion"].dumpData()
      serializable[indice]["accion"]["datos"] = self.listaAcciones[indice]["accion"]["datos"].tolist()

    return copy.deepcopy(serializable)

  def saveFile(self, ruta, data):
    with open(ruta, "w") as archivo:
      json.dump(data, archivo)

  def loadData(self, ruta, display=False):
    self.loadFile(ruta)
    self.parseDictionaryToScraper()
    if(display):
      print(f"Los datos se han cargado de {ruta}")

  def loadFile(self, ruta):
    with open(ruta, "r") as archivo:
      self.listaAcciones = json.load(archivo)

  def parseDictionaryToScraper(self):
    for indice in range(len(self.listaAcciones)):
      self.listaAcciones[indice]["accion"]["datos"] = np.array(self.listaAcciones[indice]["accion"]["datos"])
      scraper = ScraperAcciones("","")
      scraper.loadData(self.listaAcciones[indice]["accion"])
      self.listaAcciones[indice]["accion"] = scraper

In [ ]:
# @title Lista de acciones en bruto
lista_acciones = [
    {
        "nombre": "AAPL:BMV",
        "alias": "Apple"
    },
    {
        "nombre": ".INX:INDEXSP",
        "alias": "S&P 500"
    },
    {
        "nombre": "GOOGL:NASDAQ",
        "alias": "Google"
    },
    {
        "nombre": "MSFT:NASDAQ",
        "alias": "Microsoft"
    },
    {
        "nombre": "AMZN:NASDAQ",
        "alias": "Amazon"
    },
    {
        "nombre": "META:NASDAQ",
        "alias": "Meta"
    },
    {
        "nombre": "TSLA:NASDAQ",
        "alias": "Tesla"
    },
    {
        "nombre": "NVDA:NASDAQ",
        "alias": "NVIDIA"
    },
    {
        "nombre": "ADBE:NASDAQ",
        "alias": "Adobe Systems Incorporated"
    },
    {
        "nombre": "INTC:NASDAQ",
        "alias": "Intel Corporation"
    },
    {
        "nombre": "BAC:NYSE",
        "alias": "Bank of America"
    },
    {
        "nombre": "OSPTX:INDEXTSI",
        "alias": "S&P/TSX Composite Index"
    },
    {
        "nombre": "TOPIX:INDEXTOPIX",
        "alias": "TOPIX"
    },
    {
        "nombre": "SP400:INDEXSP",
        "alias": "S&P 400"
    },
    {
        "nombre": "BRK.A:NYSE",
        "alias": "Berkshire Hathaway"
    },
    {
        "nombre": "V:NYSE",
        "alias": "Visa"
    },
    {
        "nombre": "PYPL:NASDAQ",
        "alias": "PayPal"
    },
    {
        "nombre": "KOF:NYSE",
        "alias": "Coca-Cola FEMSA"
    },
    {
        "nombre": "NESN:SWX",
        "alias": "Nestlé"
    },
    {
        "nombre": "KIMBERA:BMV",
        "alias": "Kimberly-Clark de México"
    },
    {
        "nombre": "GE:BMV",
        "alias": "General Electric"
    },
    {
        "nombre": "ALFAA:BMV",
        "alias": "Alfa"
    },
    {
        "nombre": "WALMEX:BMV",
        "alias": "Walmart de México y Centroamérica"
    },
    {
        "nombre": "CEMEXCPO:BMV",
        "alias": "Cemex"
    },
    {
        "nombre": ".IXIC:INDEXNASDAQ",
        "alias": "Nasdaq Composite"
    },
    {
        "nombre": ".DJI:INDEXDJX",
        "alias": "Promedio Industrial Dow Jones"
    },
    {
        "nombre": "QQQ:NASDAQ",
        "alias": "Invesco QQQ Trust Series 1"
    },
    {
        "nombre": "HPE:NYSE",
        "alias": "Hewlett Packard Enterprise"
    },
    {
        "nombre": "SONE:STO",
        "alias": "Sonetel AB (publ)"
    },
    {
        "nombre": "QLYS:NASDAQ",
        "alias": "Qualys"
    },
    {
        "nombre": "SX5R:INDEXSTOXX",
        "alias": "STOXX Europe 50"
    },
    {
        "nombre": "INDI:INDEXBME",
        "alias": "IBEX 35"
    },
    {
        "nombre": "OEX:INDEXCBOE",
        "alias": "S&P 100"
    },
    {
        "nombre": "NFLX:NASDAQ",
        "alias": "Netflix"
    },
    {
        "nombre": "LOGN:SWX",
        "alias": "Logitech"
    },
    {
        "nombre": "VMW:NYSE",
        "alias": "VMware"
    },
    {
        "nombre": "2357:TPE",
        "alias": "ASUS"
    },
    {
        "nombre": "RUT:INDEXRUSSELL",
        "alias": "Índice Russell 2000"
    },
    {
        "nombre": "VIX:INDEXCBOE",
        "alias": "VIX"
    },
    {
        "nombre": "NEMAKA:BMV",
        "alias": "Nemak"
    },
    {
        "nombre": "MU:NASDAQ",
        "alias": "Micron Technology"
    },
    {
        "nombre": "T:NYSE",
        "alias": "AT&T"
    },
    {
        "nombre": "SQ:NYSE",
        "alias": "Square"
    },
    {
        "nombre": "QCOM:NASDAQ",
        "alias": "Qualcomm"
    },
    {
        "nombre": "XOM:NYSE",
        "alias": "ExxonMobil"
    },
    {
        "nombre": "JNJ:NYSE",
        "alias": "Johnson & Johnson"
    },
    {
        "nombre": "VZ:NYSE",
        "alias": "Verizon Communications"
    },
    {
        "nombre": "BIDU:NASDAQ",
        "alias": "Baidu"
    },
    {
        "nombre": "JPM:NYSE",
        "alias": "JP Morgan Chase"
    },
    {
        "nombre": "WFC:NYSE",
        "alias": "Wells Fargo"
    },
    {
        "nombre": "888:LON",
        "alias": "888 Holdings"
    },
    {
        "nombre": "FUNO11:BMV",
        "alias": "Fibra Uno Administracion SA de CV"
    },
    {
        "nombre": "SNAP:NYSE",
        "alias": "Snap Inc."
    },
    {
        "nombre": "CSCO:NASDAQ",
        "alias": "Cisco Systems"
    },
    {
        "nombre": "DAX:INDEXDB",
        "alias": "DAX"
    },
    {
        "nombre": "UKX:INDEXFTSE",
        "alias": "FTSE 100"
    },
    {
        "nombre": "PX1:INDEXEURO",
        "alias": "CAC 40"
    },
    {
        "nombre": "SX5E:INDEXSTOXX",
        "alias": "EURO STOXX 50"
    },
    {
        "nombre": "NI225:INDEXNIKKEI",
        "alias": "Nikkei 225"
    },
    {
        "nombre": "000001:SHA",
        "alias": "SSE Composite Index"
    },
    {
        "nombre": "HSI:INDEXHANGSENG",
        "alias": "Hang Seng Index"
    },
    {
        "nombre": "SENSEX:INDEXBOM",
        "alias": "BSE Sensex"
    },
    {
        "nombre": "NAFTRACISHRS:BMV",
        "alias": "iShares NAFTRAC"
    },
    {
        "nombre": "GMEXICOB:BMV",
        "alias": "Grupo México"
    },
    {
        "nombre": "BBAJIOO:BMV",
        "alias": "Banco del Bajío"
    },
    {
        "nombre": "GFNORTEO:BMV",
        "alias": "Grupo Financiero Banorte"
    },
    {
        "nombre": "BIMBOA:BMV",
        "alias": "Grupo Bimbo"
    },
    {
        "nombre": "MEGACPO:BMV",
        "alias": "Megacable Comunicaciones"
    },
    {
        "nombre": "CHDRAUIB:BMV",
        "alias": "Chedraui"
    },
    {
        "nombre": "GRUMAB:BMV",
        "alias": "Maseca"
    },
    {
        "nombre": "LACOMERUBC:BMV",
        "alias": "Grupo La Comer"
    },
    {
        "nombre": "HERDEZ:BMV",
        "alias": "Grupo Herdez"
    },
    {
        "nombre": "VISTAA:BMV",
        "alias": "Vista Energy SAB de CV"
    },
    {
        "nombre": "LIVEPOLC-1:BMV",
        "alias": "El Puerto de Liverpool"
    },
    {
        "nombre": "FIBRAMQ12:BMV",
        "alias": "FIBRA Macquarie"
    },
    {
        "nombre": "TERRA13:BMV",
        "alias": "PLA Administradora Industrial S de RL de CV"
    },
    {
        "nombre": "ALPEKA:BMV",
        "alias": "Alpek"
    },
    {
        "nombre": "DANHOS13:BMV",
        "alias": "Fibra Danhos"
    },
    {
        "nombre": "AGUA:BMV",
        "alias": "Grupo Rotoplas S.A.B. de C.V."
    },
    {
        "nombre": "VALUEGFO:BMV",
        "alias": "Value Grupo Financiero SAB de CV"
    },
]
random.shuffle(lista_acciones)

In [ ]:
# @title Lista de acciones en bruto de BMV Capitales
lista_acciones_bmv_capitales = [
    {
        "nombre": "AC",
        "alias": ""
    },
    {
        "nombre": "ACTINVR",
        "alias": ""
    },
    {
        "nombre": "AGUA",
        "alias": ""
    },
    {
        "nombre": "ALFA",
        "alias": ""
    },
    {
        "nombre": "ALPEK",
        "alias": ""
    },
    {
        "nombre": "ALSEA",
        "alias": ""
    },
    {
        "nombre": "ALTERNA",
        "alias": ""
    },
    {
        "nombre": "AMX",
        "alias": ""
    },
    {
        "nombre": "ANGELD",
        "alias": ""
    },
    {
        "nombre": "ARA",
        "alias": ""
    },
    {
        "nombre": "ASUR",
        "alias": ""
    },
    {
        "nombre": "AUTLAN",
        "alias": ""
    },
    {
        "nombre": "AXTEL",
        "alias": ""
    },
    {
        "nombre": "BACHOCO",
        "alias": ""
    },
    {
        "nombre": "BBAJIO",
        "alias": ""
    },
    {
        "nombre": "BBVA",
        "alias": ""
    },
    {
        "nombre": "BEVIDES",
        "alias": ""
    },
    {
        "nombre": "BIMBO",
        "alias": ""
    },
    {
        "nombre": "BOLSA",
        "alias": ""
    },
    {
        "nombre": "CADU",
        "alias": ""
    },
    {
        "nombre": "CEMEX",
        "alias": ""
    },
    {
        "nombre": "CETETRC",
        "alias": ""
    },
    {
        "nombre": "CHDRAUI",
        "alias": ""
    },
    {
        "nombre": "CHNTRAC",
        "alias": ""
    },
    {
        "nombre": "CIE",
        "alias": ""
    },
    {
        "nombre": "CMOCTEZ",
        "alias": ""
    },
    {
        "nombre": "CMR",
        "alias": ""
    },
    {
        "nombre": "COLLADO",
        "alias": ""
    },
    {
        "nombre": "CTAXTEL",
        "alias": ""
    },
    {
        "nombre": "CUERVO",
        "alias": ""
    },
    {
        "nombre": "CULTIBA",
        "alias": ""
    },
    {
        "nombre": "CYDSASA",
        "alias": ""
    },
    {
        "nombre": "DIABLOI",
        "alias": ""
    },
    {
        "nombre": "DLRTRAC",
        "alias": ""
    },
    {
        "nombre": "ELEKTRA",
        "alias": ""
    },
    {
        "nombre": "ESGMEX",
        "alias": ""
    },
    {
        "nombre": "FEMSA",
        "alias": ""
    },
    {
        "nombre": "FEMSA",
        "alias": ""
    },
    {
        "nombre": "FIBRATC",
        "alias": ""
    },
    {
        "nombre": "FINDEP",
        "alias": ""
    },
    {
        "nombre": "FRAGUA",
        "alias": ""
    },
    {
        "nombre": "FRES",
        "alias": ""
    },
    {
        "nombre": "GAP",
        "alias": ""
    },
    {
        "nombre": "GBM",
        "alias": ""
    },
    {
        "nombre": "GCARSO",
        "alias": ""
    },
    {
        "nombre": "GCC",
        "alias": ""
    },
    {
        "nombre": "GENIUS",
        "alias": ""
    },
    {
        "nombre": "GENTERA",
        "alias": ""
    },
    {
        "nombre": "GFINBUR",
        "alias": ""
    },
    {
        "nombre": "GFNORTE",
        "alias": ""
    },
    {
        "nombre": "GICSA",
        "alias": ""
    },
    {
        "nombre": "GIGANTE",
        "alias": ""
    },
    {
        "nombre": "GISSA",
        "alias": ""
    },
    {
        "nombre": "GMEXICO",
        "alias": ""
    },
    {
        "nombre": "GMXT",
        "alias": ""
    },
    {
        "nombre": "GPROFUT",
        "alias": ""
    },
    {
        "nombre": "GRUMA",
        "alias": ""
    },
    {
        "nombre": "HCITY",
        "alias": ""
    },
    {
        "nombre": "HERDEZ",
        "alias": ""
    },
    {
        "nombre": "HOMEX",
        "alias": ""
    },
    {
        "nombre": "HOTEL",
        "alias": ""
    },
    {
        "nombre": "ICH",
        "alias": ""
    },
    {
        "nombre": "INVEX",
        "alias": ""
    },
    {
        "nombre": "IVVPESO",
        "alias": ""
    },
    {
        "nombre": "KIMBER",
        "alias": ""
    },
    {
        "nombre": "KOF",
        "alias": ""
    },
    {
        "nombre": "KUO",
        "alias": ""
    },
    {
        "nombre": "LAB",
        "alias": ""
    },
    {
        "nombre": "LACOMER",
        "alias": ""
    },
    {
        "nombre": "LASITE",
        "alias": ""
    },
    {
        "nombre": "LIVEPOL",
        "alias": ""
    },
    {
        "nombre": "LIVEPOL",
        "alias": ""
    },
    {
        "nombre": "M10TRAC",
        "alias": ""
    },
    {
        "nombre": "MEDICA",
        "alias": ""
    },
    {
        "nombre": "MEGA",
        "alias": ""
    },
    {
        "nombre": "MEXTRAC",
        "alias": ""
    },
    {
        "nombre": "MFRISCO",
        "alias": ""
    },
    {
        "nombre": "MINSA",
        "alias": ""
    },
    {
        "nombre": "NAFTRAC",
        "alias": ""
    },
    {
        "nombre": "NEMAK",
        "alias": ""
    },
    {
        "nombre": "OMA",
        "alias": ""
    },
    {
        "nombre": "ORBIA",
        "alias": ""
    },
    {
        "nombre": "PASA",
        "alias": ""
    },
    {
        "nombre": "PE&OLES",
        "alias": ""
    },
    {
        "nombre": "PINFRA",
        "alias": ""
    },
    {
        "nombre": "PINFRA",
        "alias": ""
    },
    {
        "nombre": "POSADAS",
        "alias": ""
    },
    {
        "nombre": "PSOTRAC",
        "alias": ""
    },
    {
        "nombre": "Q",
        "alias": ""
    },
    {
        "nombre": "QVGMEX",
        "alias": ""
    },
    {
        "nombre": "R",
        "alias": ""
    },
    {
        "nombre": "SAN",
        "alias": ""
    },
    {
        "nombre": "SIMEC",
        "alias": ""
    },
    {
        "nombre": "SITES1",
        "alias": ""
    },
    {
        "nombre": "SORIANA",
        "alias": ""
    },
    {
        "nombre": "SPORT",
        "alias": ""
    },
    {
        "nombre": "TEAK",
        "alias": ""
    },
    {
        "nombre": "TLEVISA",
        "alias": ""
    },
    {
        "nombre": "TMM",
        "alias": ""
    },
    {
        "nombre": "TRAXION",
        "alias": ""
    },
    {
        "nombre": "UDITRAC",
        "alias": ""
    },
    {
        "nombre": "VALUEGF",
        "alias": ""
    },
    {
        "nombre": "VASCONI",
        "alias": ""
    },
    {
        "nombre": "VESTA",
        "alias": ""
    },
    {
        "nombre": "VINTE",
        "alias": ""
    },
    {
        "nombre": "VISTA",
        "alias": ""
    },
    {
        "nombre": "VMEX",
        "alias": ""
    },
    {
        "nombre": "VOLAR",
        "alias": ""
    },
    {
        "nombre": "WALMEX",
        "alias": ""
    },
]

In [ ]:
# @title Lista de acciones en bruto de BMV Global
lista_acciones_bmv_global = [
    {
        "nombre": "1810",
        "alias": ""
    },
    {
        "nombre": "AA1",
        "alias": ""
    },
    {
        "nombre": "AAL",
        "alias": ""
    },
    {
        "nombre": "AAPL",
        "alias": ""
    },
    {
        "nombre": "AAXJ",
        "alias": ""
    },
    {
        "nombre": "ABNB",
        "alias": ""
    },
    {
        "nombre": "ABT",
        "alias": ""
    },
    {
        "nombre": "ACWI",
        "alias": ""
    },
    {
        "nombre": "ACWV",
        "alias": ""
    },
    {
        "nombre": "ACWX",
        "alias": ""
    },
    {
        "nombre": "AD",
        "alias": ""
    },
    {
        "nombre": "ADBE",
        "alias": ""
    },
    {
        "nombre": "ADM",
        "alias": ""
    },
    {
        "nombre": "ADSK",
        "alias": ""
    },
    {
        "nombre": "AFRM",
        "alias": ""
    },
    {
        "nombre": "AGG",
        "alias": ""
    },
    {
        "nombre": "AI1",
        "alias": ""
    },
    {
        "nombre": "AIG",
        "alias": ""
    },
    {
        "nombre": "AIQ",
        "alias": ""
    },
    {
        "nombre": "ALB",
        "alias": ""
    },
    {
        "nombre": "ALLY1",
        "alias": ""
    },
    {
        "nombre": "ALTR",
        "alias": ""
    },
    {
        "nombre": "AMAT",
        "alias": ""
    },
    {
        "nombre": "AMC",
        "alias": ""
    },
    {
        "nombre": "AMD",
        "alias": ""
    },
    {
        "nombre": "AMZN",
        "alias": ""
    },
    {
        "nombre": "ANSS",
        "alias": ""
    },
    {
        "nombre": "ARDX",
        "alias": ""
    },
    {
        "nombre": "ASML",
        "alias": ""
    },
    {
        "nombre": "ASML1",
        "alias": ""
    },
    {
        "nombre": "ATER",
        "alias": ""
    },
    {
        "nombre": "AVGO",
        "alias": ""
    },
    {
        "nombre": "AXP",
        "alias": ""
    },
    {
        "nombre": "AZN",
        "alias": ""
    },
    {
        "nombre": "AZO",
        "alias": ""
    },
    {
        "nombre": "BA",
        "alias": ""
    },
    {
        "nombre": "BABA",
        "alias": ""
    },
    {
        "nombre": "BAC",
        "alias": ""
    },
    {
        "nombre": "BAH",
        "alias": ""
    },
    {
        "nombre": "BAP",
        "alias": ""
    },
    {
        "nombre": "BB",
        "alias": ""
    },
    {
        "nombre": "BBIL",
        "alias": ""
    },
    {
        "nombre": "BIDU",
        "alias": ""
    },
    {
        "nombre": "BIL",
        "alias": ""
    },
    {
        "nombre": "BKNG",
        "alias": ""
    },
    {
        "nombre": "BLK",
        "alias": ""
    },
    {
        "nombre": "BND",
        "alias": ""
    },
    {
        "nombre": "BNGO",
        "alias": ""
    },
    {
        "nombre": "BNTX",
        "alias": ""
    },
    {
        "nombre": "BOTZ",
        "alias": ""
    },
    {
        "nombre": "BP",
        "alias": ""
    },
    {
        "nombre": "BRKB",
        "alias": ""
    },
    {
        "nombre": "BSV",
        "alias": ""
    },
    {
        "nombre": "BSY",
        "alias": ""
    },
    {
        "nombre": "BTEC1",
        "alias": ""
    },
    {
        "nombre": "BX",
        "alias": ""
    },
    {
        "nombre": "BYND",
        "alias": ""
    },
    {
        "nombre": "C",
        "alias": ""
    },
    {
        "nombre": "CBU0",
        "alias": ""
    },
    {
        "nombre": "CBU7",
        "alias": ""
    },
    {
        "nombre": "CCL1",
        "alias": ""
    },
    {
        "nombre": "CDNS",
        "alias": ""
    },
    {
        "nombre": "CHPT",
        "alias": ""
    },
    {
        "nombre": "CHWY",
        "alias": ""
    },
    {
        "nombre": "CIBR",
        "alias": ""
    },
    {
        "nombre": "CLF",
        "alias": ""
    },
    {
        "nombre": "CNDX",
        "alias": ""
    },
    {
        "nombre": "COF",
        "alias": ""
    },
    {
        "nombre": "COIN",
        "alias": ""
    },
    {
        "nombre": "COMT",
        "alias": ""
    },
    {
        "nombre": "COST",
        "alias": ""
    },
    {
        "nombre": "CPB",
        "alias": ""
    },
    {
        "nombre": "CRM",
        "alias": ""
    },
    {
        "nombre": "CRSR",
        "alias": ""
    },
    {
        "nombre": "CRUD",
        "alias": ""
    },
    {
        "nombre": "CRWD",
        "alias": ""
    },
    {
        "nombre": "CSCO",
        "alias": ""
    },
    {
        "nombre": "CSPX",
        "alias": ""
    },
    {
        "nombre": "CSPXX",
        "alias": ""
    },
    {
        "nombre": "CSU",
        "alias": ""
    },
    {
        "nombre": "CTRA",
        "alias": ""
    },
    {
        "nombre": "CVNA",
        "alias": ""
    },
    {
        "nombre": "CVS",
        "alias": ""
    },
    {
        "nombre": "CVX",
        "alias": ""
    },
    {
        "nombre": "CXSE",
        "alias": ""
    },
    {
        "nombre": "CYBR",
        "alias": ""
    },
    {
        "nombre": "DAL",
        "alias": ""
    },
    {
        "nombre": "DAVA",
        "alias": ""
    },
    {
        "nombre": "DB",
        "alias": ""
    },
    {
        "nombre": "DDD",
        "alias": ""
    },
    {
        "nombre": "DE",
        "alias": ""
    },
    {
        "nombre": "DHR",
        "alias": ""
    },
    {
        "nombre": "DIA",
        "alias": ""
    },
    {
        "nombre": "DIS",
        "alias": ""
    },
    {
        "nombre": "DKNG1",
        "alias": ""
    },
    {
        "nombre": "DLTR",
        "alias": ""
    },
    {
        "nombre": "DOG",
        "alias": ""
    },
    {
        "nombre":"DRIP",
        "alias": ""
    },
    {
        "nombre":"DTLA",
        "alias": ""
    },
    {
        "nombre":"DVN",
        "alias": ""
    },
    {
        "nombre":"DVY",
        "alias": ""
    },
    {
        "nombre":"DXCM",
        "alias": ""
    },
    {
        "nombre":"EBIZ",
        "alias": ""
    },
    {
        "nombre":"ECAR",
        "alias": ""
    },
    {
        "nombre":"EDC",
        "alias": ""
    },
    {
        "nombre":"EDZ",
        "alias": ""
    },
    {
        "nombre":"EEM",
        "alias": ""
    },
    {
        "nombre":"EFV",
        "alias": ""
    },
    {
        "nombre":"EIDO",
        "alias": ""
    },
    {
        "nombre":"EIMI",
        "alias": ""
    },
    {
        "nombre":"EL",
        "alias": ""
    },
    {
        "nombre":"EMB",
        "alias": ""
    },
    {
        "nombre":"EMBMXX",
        "alias": ""
    },
    {
        "nombre":"EMUU",
        "alias": ""
    },
    {
        "nombre":"ENPH",
        "alias": ""
    },
    {
        "nombre":"EPP",
        "alias": ""
    },
    {
        "nombre":"EQIX",
        "alias": ""
    },
    {
        "nombre":"ETSY",
        "alias": ""
    },
    {
        "nombre":"EWC",
        "alias": ""
    },
    {
        "nombre":"EWG",
        "alias": ""
    },
    {
        "nombre":"EWJ",
        "alias": ""
    },
    {
        "nombre":"EWL",
        "alias": ""
    },
    {
        "nombre":"EWM",
        "alias": ""
    },
    {
        "nombre":"EWS",
        "alias": ""
    },
    {
        "nombre": "EWT",
        "alias": ""
    },
    {
        "nombre": "EWU",
        "alias": ""
    },
    {
        "nombre": "EWW",
        "alias": ""
    },
    {
        "nombre": "EWY",
        "alias": ""
    },
    {
        "nombre": "EWZ",
        "alias": ""
    },
    {
        "nombre": "EXC",
        "alias": ""
    },
    {
        "nombre": "EXPE",
        "alias": ""
    },
    {
        "nombre": "EZA",
        "alias": ""
    },
    {
        "nombre": "F",
        "alias": ""
    },
    {
        "nombre": "FAS",
        "alias": ""
    },
    {
        "nombre": "FAZ",
        "alias": ""
    },
    {
        "nombre": "FBT",
        "alias": ""
    },
    {
        "nombre": "FCEL",
        "alias": ""
    },
    {
        "nombre": "FCX",
        "alias": ""
    },
    {
        "nombre": "FDN",
        "alias": ""
    },
    {
        "nombre": "FDX",
        "alias": ""
    },
    {
        "nombre": "FE",
        "alias": ""
    },
    {
        "nombre": "FLAX",
        "alias": ""
    },
    {
        "nombre": "FLBR",
        "alias": ""
    },
    {
        "nombre": "FLGR",
        "alias": ""
    },
    {
        "nombre": "FLMX",
        "alias": ""
    },
    {
        "nombre": "FLOA",
        "alias": ""
    },
    {
        "nombre": "FRMXNX",
        "alias": ""
    },
    {
        "nombre": "FSLY",
        "alias": ""
    },
    {
        "nombre": "FTNT",
        "alias": ""
    },
    {
        "nombre": "FUBO",
        "alias": ""
    },
    {
        "nombre": "FXE",
        "alias": ""
    },
    {
        "nombre": "FXI",
        "alias": ""
    },
    {
        "nombre": "GD",
        "alias": ""
    },
    {
        "nombre": "GDX",
        "alias": ""
    },
    {
        "nombre": "GDXJ",
        "alias": ""
    },
    {
        "nombre": "GE",
        "alias": ""
    },
    {
        "nombre": "GEHC",
        "alias": ""
    },
    {
        "nombre": "GH",
        "alias": ""
    },
    {
        "nombre": "GLD",
        "alias": ""
    },
    {
        "nombre": "GM",
        "alias": ""
    },
    {
        "nombre": "GME",
        "alias": ""
    },
    {
        "nombre": "GMED",
        "alias": ""
    },
    {
        "nombre": "GNOM",
        "alias": ""
    },
    {
        "nombre": "GOLD",
        "alias": ""
    },
    {
        "nombre": "GOOG",
        "alias": ""
    },
    {
        "nombre": "GOOGL",
        "alias": ""
    },
    {
        "nombre": "GS",
        "alias": ""
    },
    {
        "nombre": "GUSH",
        "alias": ""
    },
    {
        "nombre": "HAL",
        "alias": ""
    },
    {
        "nombre": "HD",
        "alias": ""
    },
    {
        "nombre": "HDV",
        "alias": ""
    },
    {
        "nombre": "HEAL",
        "alias": ""
    },
    {
        "nombre": "HLQD",
        "alias": ""
    },
    {
        "nombre": "HLTH",
        "alias": ""
    },
    {
        "nombre": "HOOD",
        "alias": ""
    },
    {
        "nombre": "HPQ",
        "alias": ""
    },
    {
        "nombre": "HYG",
        "alias": ""
    },
    {
        "nombre": "IAU",
        "alias": ""
    },
    {
        "nombre": "IB01",
        "alias": ""
    },
    {
        "nombre": "IB1MXX",
        "alias": ""
    },
    {
        "nombre": "IBLC",
        "alias": ""
    },
    {
        "nombre": "IBM",
        "alias": ""
    },
    {
        "nombre": "IBTA",
        "alias": ""
    },
    {
        "nombre": "IBTMXX",
        "alias": ""
    },
    {
        "nombre": "ICHN",
        "alias": ""
    },
    {
        "nombre": "ICLN",
        "alias": ""
    },
    {
        "nombre": "ICOM",
        "alias": ""
    },
    {
        "nombre": "IDTM",
        "alias": ""
    },
    {
        "nombre": "IDTXX",
        "alias": ""
    },
    {
        "nombre": "IEF",
        "alias": ""
    },
    {
        "nombre": "IEFA",
        "alias": ""
    },
    {
        "nombre": "IEI",
        "alias": ""
    },
    {
        "nombre": "IEMG",
        "alias": ""
    },
    {
        "nombre": "IEUR",
        "alias": ""
    },
    {
        "nombre": "IEV",
        "alias": ""
    },
    {
        "nombre": "IGIB",
        "alias": ""
    },
    {
        "nombre": "IGLB",
        "alias": ""
    },
    {
        "nombre": "IGOV",
        "alias": ""
    },
    {
        "nombre": "IHI",
        "alias": ""
    },
    {
        "nombre": "IHYA",
        "alias": ""
    },
    {
        "nombre": "IJPD",
        "alias": ""
    },
    {
        "nombre": "IMBA",
        "alias": ""
    },
    {
        "nombre": "INDA",
        "alias": ""
    },
    {
        "nombre": "INDY",
        "alias": ""
    },
    {
        "nombre": "INO",
        "alias": ""
    },
    {
        "nombre": "INTC",
        "alias": ""
    },
    {
        "nombre": "INTU",
        "alias": ""
    },
    {
        "nombre": "ISAC",
        "alias": ""
    },
    {
        "nombre": "ISRG",
        "alias": ""
    },
    {
        "nombre": "ITA",
        "alias": ""
    },
    {
        "nombre": "ITEK",
        "alias": ""
    },
    {
        "nombre": "ITOT",
        "alias": ""
    },
    {
        "nombre": "IUAA",
        "alias": ""
    },
    {
        "nombre": "IUCD",
        "alias": ""
    },
    {
        "nombre": "IUCM",
        "alias": ""
    },
    {
        "nombre": "IUCS",
        "alias": ""
    },
    {
        "nombre": "IUES",
        "alias": ""
    },
    {
        "nombre": "IUFS",
        "alias": ""
    },
    {
        "nombre": "IUHC",
        "alias": ""
    },
    {
        "nombre": "IUIS",
        "alias": ""
    },
    {
        "nombre": "IUIT",
        "alias": ""
    },
    {
        "nombre": "IUQA",
        "alias": ""
    },
    {
        "nombre": "IVV",
        "alias": ""
    },
    {
        "nombre": "IWDA",
        "alias": ""
    },
    {
        "nombre": "IWM",
        "alias": ""
    },
    {
        "nombre": "IWVL",
        "alias": ""
    },
    {
        "nombre": "IXC",
        "alias": ""
    },
    {
        "nombre": "IXJ",
        "alias": ""
    },
    {
        "nombre": "IXN",
        "alias": ""
    },
    {
        "nombre": "IYW",
        "alias": ""
    },
    {
        "nombre": "IYY",
        "alias": ""
    },
    {
        "nombre": "JBSAY",
        "alias": ""
    },
    {
        "nombre": "JD",
        "alias": ""
    },
    {
        "nombre": "JDST",
        "alias": ""
    },
    {
        "nombre": "JNJ",
        "alias": ""
    },
    {
        "nombre": "JNUG",
        "alias": ""
    },
    {
        "nombre": "JPEA",
        "alias": ""
    },
    {
        "nombre": "JPM",
        "alias": ""
    },
    {
        "nombre": "JWN",
        "alias": ""
    },
    {
        "nombre": "KHC",
        "alias": ""
    },
    {
        "nombre": "KMI",
        "alias": ""
    },
    {
        "nombre": "KO",
        "alias": ""
    },
    {
        "nombre": "KRE",
        "alias": ""
    },
    {
        "nombre": "KWEB",
        "alias": ""
    },
    {
        "nombre": "LABD",
        "alias": ""
    },
    {
        "nombre": "LABU",
        "alias": ""
    },
    {
        "nombre": "LCID",
        "alias": ""
    },
    {
        "nombre": "LEMB",
        "alias": ""
    },
    {
        "nombre": "LIT",
        "alias": ""
    },
    {
        "nombre": "LLY",
        "alias": ""
    },
    {
        "nombre": "LMT",
        "alias": ""
    },
    {
        "nombre": "LQDA",
        "alias": ""
    },
    {
        "nombre": "LQDH",
        "alias": ""
    },
    {
        "nombre": "LQDMX",
        "alias": ""
    },
    {
        "nombre": "LULU",
        "alias": ""
    },
    {
        "nombre": "LUV",
        "alias": ""
    },
    {
        "nombre": "LVHD",
        "alias": ""
    },
    {
        "nombre": "LVS",
        "alias": ""
    },
    {
        "nombre": "LYFT",
        "alias": ""
    },
    {
        "nombre": "M",
        "alias": ""
    },
    {
        "nombre": "MARA",
        "alias": ""
    },
    {
        "nombre": "MAT",
        "alias": ""
    },
    {
        "nombre": "MB3M",
        "alias": ""
    },
    {
        "nombre": "MBIL",
        "alias": ""
    },
    {
        "nombre": "MC",
        "alias": ""
    },
    {
        "nombre": "MCD",
        "alias": ""
    },
    {
        "nombre": "MCHI",
        "alias": ""
    },
    {
        "nombre": "MCHP",
        "alias": ""
    },
    {
        "nombre": "MELI",
        "alias": ""
    },
    {
        "nombre": "META",
        "alias": ""
    },
    {
        "nombre": "MGM",
        "alias": ""
    },
    {
        "nombre": "MMM",
        "alias": ""
    },
    {
        "nombre": "MNST",
        "alias": ""
    },
    {
        "nombre": "MO",
        "alias": ""
    },
    {
        "nombre": "MRK",
        "alias": ""
    },
    {
        "nombre": "MRNA",
        "alias": ""
    },
    {
        "nombre": "MS",
        "alias": ""
    },
    {
        "nombre": "MSFT",
        "alias": ""
    },
    {
        "nombre": "MSTR",
        "alias": ""
    },
    {
        "nombre": "MT",
        "alias": ""
    },
    {
        "nombre": "MU",
        "alias": ""
    },
    {
        "nombre": "NCLH",
        "alias": ""
    },
    {
        "nombre": "NESN",
        "alias": ""
    },
    {
        "nombre": "NFLX",
        "alias": ""
    },
    {
        "nombre": "NIO",
        "alias": ""
    },
    {
        "nombre": "NKE",
        "alias": ""
    },
    {
        "nombre": "NKLA",
        "alias": ""
    },
    {
        "nombre": "NOK",
        "alias": ""
    },
    {
        "nombre": "NOVOB",
        "alias": ""
    },
    {
        "nombre": "NOVT",
        "alias": ""
    },
    {
        "nombre": "NOWW",
        "alias": ""
    },
    {
        "nombre": "NTDOY",
        "alias": ""
    },
    {
        "nombre": "NU",
        "alias": ""
    },
    {
        "nombre": "NVAX",
        "alias": ""
    },
    {
        "nombre": "NVDA",
        "alias": ""
    },
    {
        "nombre": "NXTG",
        "alias": ""
    },
    {
        "nombre": "OGN",
        "alias": ""
    },
    {
        "nombre": "ON",
        "alias": ""
    },
    {
        "nombre": "OPEN1",
        "alias": ""
    },
    {
        "nombre": "ORCL",
        "alias": ""
    },
    {
        "nombre": "ORLY",
        "alias": ""
    },
    {
        "nombre": "OXY1",
        "alias": ""
    },
    {
        "nombre": "PANW",
        "alias": ""
    },
    {
        "nombre": "PBR",
        "alias": ""
    },
    {
        "nombre": "PDD",
        "alias": ""
    },
    {
        "nombre": "PEP",
        "alias": ""
    },
    {
        "nombre": "PFE",
        "alias": ""
    },
    {
        "nombre": "PFF",
        "alias": ""
    },
    {
        "nombre": "PG",
        "alias": ""
    },
    {
        "nombre": "PII",
        "alias": ""
    },
    {
        "nombre": "PLTR",
        "alias": ""
    },
    {
        "nombre": "PLUG",
        "alias": ""
    },
    {
        "nombre": "PM",
        "alias": ""
    },
    {
        "nombre": "POOL",
        "alias": ""
    },
    {
        "nombre": "PR",
        "alias": ""
    },
    {
        "nombre": "PROK",
        "alias": ""
    },
    {
        "nombre": "PSQ",
        "alias": ""
    },
    {
        "nombre": "PTON",
        "alias": ""
    },
    {
        "nombre": "PYPL",
        "alias": ""
    },
    {
        "nombre": "QCLN",
        "alias": ""
    },
    {
        "nombre": "QCOM",
        "alias": ""
    },
    {
        "nombre": "QLD",
        "alias": ""
    },
    {
        "nombre": "QQQ",
        "alias": ""
    },
    {
        "nombre": "QS",
        "alias": ""
    },
    {
        "nombre": "QUAL",
        "alias": ""
    },
    {
        "nombre": "QYLD",
        "alias": ""
    },
    {
        "nombre": "RACE",
        "alias": ""
    },
    {
        "nombre": "RBLX",
        "alias": ""
    },
    {
        "nombre": "RBOT",
        "alias": ""
    },
    {
        "nombre": "RCL",
        "alias": ""
    },
    {
        "nombre": "RIG",
        "alias": ""
    },
    {
        "nombre": "RIO",
        "alias": ""
    },
    {
        "nombre": "RIOT",
        "alias": ""
    },
    {
        "nombre": "RIVN",
        "alias": ""
    },
    {
        "nombre": "RNRG",
        "alias": ""
    },
    {
        "nombre": "ROG",
        "alias": ""
    },
    {
        "nombre": "ROKU",
        "alias": ""
    },
    {
        "nombre": "RSP",
        "alias": ""
    },
    {
        "nombre": "RTX",
        "alias": ""
    },
    {
        "nombre": "RUN",
        "alias": ""
    },
    {
        "nombre": "SAND",
        "alias": ""
    },
    {
        "nombre": "SASU",
        "alias": ""
    },
    {
        "nombre": "SBUX",
        "alias": ""
    },
    {
        "nombre": "SCHW",
        "alias": ""
    },
    {
        "nombre": "SDHA",
        "alias": ""
    },
    {
        "nombre": "SDIA",
        "alias": ""
    },
    {
        "nombre": "SDIV",
        "alias": ""
    },
    {
        "nombre": "SDMXX",
        "alias": ""
    },
    {
        "nombre": "SE",
        "alias": ""
    },
    {
        "nombre": "SEDG",
        "alias": ""
    },
    {
        "nombre": "SGOV",
        "alias": ""
    },
    {
        "nombre": "SHEL",
        "alias": ""
    },
    {
        "nombre": "SHOP",
        "alias": ""
    },
    {
        "nombre": "SHV",
        "alias": ""
    },
    {
        "nombre": "SHY",
        "alias": ""
    },
    {
        "nombre": "SLAB",
        "alias": ""
    },
    {
        "nombre": "SLB",
        "alias": ""
    },
    {
        "nombre": "SLQD",
        "alias": ""
    },
    {
        "nombre": "SLV",
        "alias": ""
    },
    {
        "nombre": "SMFG",
        "alias": ""
    },
    {
        "nombre": "SMH",
        "alias": ""
    },
    {
        "nombre": "SMHU",
        "alias": ""
    },
    {
        "nombre": "SMSN",
        "alias": ""
    },
    {
        "nombre": "SNAP",
        "alias": ""
    },
    {
        "nombre": "SNOW",
        "alias": ""
    },
    {
        "nombre": "SNY",
        "alias": ""
    },
    {
        "nombre": "SOCL",
        "alias": ""
    },
    {
        "nombre": "SOFI",
        "alias": ""
    },
    {
        "nombre": "SOON",
        "alias": ""
    },
    {
        "nombre": "SOXL",
        "alias": ""
    },
    {
        "nombre": "SOXS",
        "alias": ""
    },
    {
        "nombre": "SOXX",
        "alias": ""
    },
    {
        "nombre": "SPCE",
        "alias": ""
    },
    {
        "nombre": "SPLG",
        "alias": ""
    },
    {
        "nombre": "SPLK",
        "alias": ""
    },
    {
        "nombre": "SPOT",
        "alias": ""
    },
    {
        "nombre": "SPXL",
        "alias": ""
    },
    {
        "nombre": "SPXS",
        "alias": ""
    },
    {
        "nombre": "SPY",
        "alias": ""
    },
    {
        "nombre": "SPY4",
        "alias": ""
    },
    {
        "nombre": "SPYD",
        "alias": ""
    },
    {
        "nombre": "SQ",
        "alias": ""
    },
    {
        "nombre": "SQQQ",
        "alias": ""
    },
    {
        "nombre": "STZ",
        "alias": ""
    },
    {
        "nombre": "SUAS",
        "alias": ""
    },
    {
        "nombre": "SXLC",
        "alias": ""
    },
    {
        "nombre": "SXLF",
        "alias": ""
    },
    {
        "nombre": "SXRJ",
        "alias": ""
    },
    {
        "nombre": "T",
        "alias": ""
    },
    {
        "nombre": "TAL",
        "alias": ""
    },
    {
        "nombre": "TAN",
        "alias": ""
    },
    {
        "nombre": "TBT",
        "alias": ""
    },
    {
        "nombre": "TCEHY",
        "alias": ""
    },
    {
        "nombre": "TDOC",
        "alias": ""
    },
    {
        "nombre": "TECL",
        "alias": ""
    },
    {
        "nombre": "TECS",
        "alias": ""
    },
    {
        "nombre": "TER",
        "alias": ""
    },
    {
        "nombre": "TFLO",
        "alias": ""
    },
    {
        "nombre": "TFRN",
        "alias": ""
    },
    {
        "nombre": "TGT",
        "alias": ""
    },
    {
        "nombre": "TIP",
        "alias": ""
    },
    {
        "nombre": "TLT",
        "alias": ""
    },
    {
        "nombre": "TMF",
        "alias": ""
    },
    {
        "nombre": "TMO",
        "alias": ""
    },
    {
        "nombre": "TMUS",
        "alias": ""
    },
    {
        "nombre": "TNA",
        "alias": ""
    },
    {
        "nombre": "TQQQ",
        "alias": ""
    },
    {
        "nombre": "TSLA",
        "alias": ""
    },
    {
        "nombre": "TSM",
        "alias": ""
    },
    {
        "nombre": "TSN",
        "alias": ""
    },
    {
        "nombre": "TTE",
        "alias": ""
    },
    {
        "nombre": "U",
        "alias": ""
    },
    {
        "nombre": "UAA",
        "alias": ""
    },
    {
        "nombre": "UAL",
        "alias": ""
    },
    {
        "nombre": "UBER",
        "alias": ""
    },
    {
        "nombre": "UL",
        "alias": ""
    },
    {
        "nombre": "UNH",
        "alias": ""
    },
    {
        "nombre": "UPST",
        "alias": ""
    },
    {
        "nombre": "URA",
        "alias": ""
    },
    {
        "nombre": "URTH",
        "alias": ""
    },
    {
        "nombre": "USB",
        "alias": ""
    },
    {
        "nombre": "USFR",
        "alias": ""
    },
    {
        "nombre": "USMV",
        "alias": ""
    },
    {
        "nombre": "USPX",
        "alias": ""
    },
    {
        "nombre": "USRI",
        "alias": ""
    },
    {
        "nombre": "USRT",
        "alias": ""
    },
    {
        "nombre": "UTSI",
        "alias": ""
    },
    {
        "nombre": "V",
        "alias": ""
    },
    {
        "nombre": "VAGU",
        "alias": ""
    },
    {
        "nombre": "VALE",
        "alias": ""
    },
    {
        "nombre": "VAPU",
        "alias": ""
    },
    {
        "nombre": "VCSH",
        "alias": ""
    },
    {
        "nombre": "VDC",
        "alias": ""
    },
    {
        "nombre": "VDST",
        "alias": ""
    },
    {
        "nombre": "VDTA",
        "alias": ""
    },
    {
        "nombre": "VEA",
        "alias": ""
    },
    {
        "nombre": "VEU",
        "alias": ""
    },
    {
        "nombre": "VFEA",
        "alias": ""
    },
    {
        "nombre": "VGK",
        "alias": ""
    },
    {
        "nombre": "VGT",
        "alias": ""
    },
    {
        "nombre": "VHYA",
        "alias": ""
    },
    {
        "nombre": "VIG",
        "alias": ""
    },
    {
        "nombre": "VIS",
        "alias": ""
    },
    {
        "nombre": "VJPA",
        "alias": ""
    },
    {
        "nombre": "VLUE",
        "alias": ""
    },
    {
        "nombre": "VMSTX",
        "alias": ""
    },
    {
        "nombre": "VNQ",
        "alias": ""
    },
    {
        "nombre": "VNQI",
        "alias": ""
    },
    {
        "nombre": "VNRA",
        "alias": ""
    },
    {
        "nombre": "VOO",
        "alias": ""
    },
    {
        "nombre": "VRM",
        "alias": ""
    },
    {
        "nombre": "VT",
        "alias": ""
    },
    {
        "nombre": "VTI",
        "alias": ""
    },
    {
        "nombre": "VTIP",
        "alias": ""
    },
    {
        "nombre": "VTRS",
        "alias": ""
    },
    {
        "nombre": "VUAA",
        "alias": ""
    },
    {
        "nombre": "VUG",
        "alias": ""
    },
    {
        "nombre": "VWCG",
        "alias": ""
    },
    {
        "nombre": "VWO",
        "alias": ""
    },
    {
        "nombre": "VWRA",
        "alias": ""
    },
    {
        "nombre": "VXUS",
        "alias": ""
    },
    {
        "nombre": "VYM",
        "alias": ""
    },
    {
        "nombre": "VZ",
        "alias": ""
    },
    {
        "nombre": "WANT",
        "alias": ""
    },
    {
        "nombre": "WBA",
        "alias": ""
    },
    {
        "nombre": "WBD",
        "alias": ""
    },
    {
        "nombre": "WE",
        "alias": ""
    },
    {
        "nombre": "WFC",
        "alias": ""
    },
    {
        "nombre": "WISH",
        "alias": ""
    },
    {
        "nombre": "WMT",
        "alias": ""
    },
    {
        "nombre": "WOOF",
        "alias": ""
    },
    {
        "nombre": "WYNN",
        "alias": ""
    },
    {
        "nombre": "X",
        "alias": ""
    },
    {
        "nombre": "XBI",
        "alias": ""
    },
    {
        "nombre": "XDEW",
        "alias": ""
    },
    {
        "nombre": "XLE",
        "alias": ""
    },
    {
        "nombre": "XLF",
        "alias": ""
    },
    {
        "nombre": "XLI",
        "alias": ""
    },
    {
        "nombre": "XLK",
        "alias": ""
    },
    {
        "nombre": "XLP",
        "alias": ""
    },
    {
        "nombre": "XLRE",
        "alias": ""
    },
    {
        "nombre": "XLU",
        "alias": ""
    },
    {
        "nombre": "XLV",
        "alias": ""
    },
    {
        "nombre": "XLY",
        "alias": ""
    },
    {
        "nombre": "XOM",
        "alias": ""
    },
    {
        "nombre": "XPEV",
        "alias": ""
    },
    {
        "nombre": "XPH",
        "alias": ""
    },
    {
        "nombre": "XSOE",
        "alias": ""
    },
    {
        "nombre": "XSVM",
        "alias": ""
    },
    {
        "nombre": "YANG",
        "alias": ""
    },
    {
        "nombre": "YINN",
        "alias": ""
    },
    {
        "nombre": "ZM",
        "alias": ""
    },
    {
        "nombre": "ZPR1",
        "alias": ""
    },
    {
        "nombre": "ZS",
        "alias": ""
    },
]

In [ ]:
# @title ProcesadorDatos: Objeto encargado de procesar los datos en bruto para convertirlos en datos de entrenamiento
class ProcesadorDatos:
  def __init__(self, datos, limiteDatos = 255, contextoSize = 0.8, trainSize = 0.8, escalado = 1000):
    self.escalado = escalado
    self.contextoSize = contextoSize
    self.trainSize = trainSize
    self.limiteDatos = limiteDatos
    self.datos = datos
    self.inicializarXY()

  def inicializarXY(self):
    self.profundidad = self.calcularProfundidad()
    self.dimX = int(self.limiteDatos * self.contextoSize)
    self.dimY = self.limiteDatos - self.dimX
    self.X = np.empty((0, self.dimX, self.profundidad))
    self.Y = np.empty((0, self.dimY, self.profundidad))
    self.acciones = list()

  def calcularProfundidad(self):
    datoBase = self.datos[0]
    accion = datoBase["accion"]
    datosAccion = accion.datos
    dimensiones = datosAccion.shape
    profundidad = dimensiones[-1]
    return profundidad

  def procesar(self):
    self.makeXY()
    corte = self.calcularPuntoCorte()
    self.partir_dataset(corte)

  def makeXY(self):
    for accion in self.datos:
      self.asignarXY(accion)

  def asignarXY(self, accion):
    accionScraper = accion["accion"]
    X, Y = self.tratar_datos(accionScraper)
    self.X = np.append(self.X, [X], axis=0)
    self.Y = np.append(self.Y, [Y], axis=0)
    self.acciones.append(accion)

  def calcularPuntoCorte(self):
    return int(self.X.shape[0]*self.trainSize)

  def partir_dataset(self, corte):
    self.X_train = self.X[0:corte] / self.escalado
    self.X_test = self.X[:-corte] / self.escalado
    self.Y_train = self.Y[0:corte] / self.escalado
    self.Y_test = self.Y[:-corte] / self.escalado
    self.trainAcciones = np.array(self.acciones[0:corte])
    self.testAcciones = np.array(self.acciones[:-corte])

  def tratar_datos(self, accion):
    datos = copy.deepcopy(accion.datos)
    train_limit = int(self.contextoSize * self.limiteDatos)
    x_dato = datos[: train_limit]
    #x_dato = self.aplanar_datos(x_dato)
    y_dato = datos[train_limit: self.limiteDatos]
    #y_dato = self.aplanar_datos(y_dato)
    return x_dato, y_dato

  def aplanar_datos(self, datos):
    datos_aplanados = np.array([])
    for dato in datos:
      datos_aplanados = np.concatenate((datos_aplanados, dato))
    return datos_aplanados

In [ ]:
# @title Funcion graficado de svg

#def graficarSvg(args):
#  paths = ""
#  for arg in args:
#    data = arg["data"]
#    color = arg["color"]
#    paths = paths + f"""
#      <path fill="none" stroke-width="2" stroke="{color}" stroke-dasharray="none" opacity="1" d="{data}">
#    </path>
#    """
#  html_code = f"""
#  <svg class="svg-datos" xmlns="http://www.w3.org/2000/svg">
#    {paths}
#  </svg>
#  <script>
#    svgs = document.getElementsByClassName("svg-datos");
#        for (let svg of svgs){{
#            path = svg.children[0];
#            x = path.getBoundingClientRect().width;
#            y = path.getBoundingClientRect().height;
#            svg.style.width = x*1.5 + "px";
#            svg.style.height = y*1.5 + "px";
#        }}
#  </script>
#  """
#
#  display(HTML(html_code))

In [ ]:
# @title Modelo entrenable
class Modelo:

  class CustomCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        clear_output()

  def __init__(self):
    self.model = None

  def setProcesador(self, procesador):
    self.procesador = procesador

  def makePresets(self):
    self.preset = [
        tf.keras.Sequential([
            tf.keras.layers.Flatten(input_shape=(self.procesador.dimX, self.procesador.profundidad)),
            tf.keras.layers.Dense(600, activation='sigmoid'),
            tf.keras.layers.Dense(800, activation='sigmoid'),
            tf.keras.layers.Dense(1000, activation='sigmoid'),
            tf.keras.layers.Dense(800, activation='sigmoid'),
            tf.keras.layers.Dense(600, activation='sigmoid'),
            tf.keras.layers.Dense(self.procesador.dimY * self.procesador.profundidad, activation='linear')
        ]),
        tf.keras.Sequential([
            tf.keras.layers.LSTM(200, input_shape=(204,2), return_sequences=True),
            tf.keras.layers.LSTM(102, return_sequences=True),
            tf.keras.layers.LSTM(50, return_sequences=True),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Reshape((51, 200)),
            tf.keras.layers.LSTM(200, input_shape=(51,200), return_sequences=True),
            tf.keras.layers.LSTM(2, return_sequences=True),
        ]),
        tf.keras.Sequential([
            tf.keras.layers.Input(shape=(self.procesador.dimX, self.procesador.profundidad, 1)),
            #tf.keras.layers.Conv2D(32, (3, 3), activation='sigmoid', padding='same', kernel_initializer='glorot_uniform'),
            #tf.keras.layers.Dropout(0.5),
            #tf.keras.layers.MaxPooling2D((2, 2)),
            #tf.keras.layers.Conv2D(64, (3, 3), activation='sigmoid', padding='same'),
            #tf.keras.layers.MaxPooling2D((2, 2)),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(102, activation='linear')
        ]),
        tf.keras.Sequential([
            tf.keras.layers.Input(shape=(self.procesador.dimX)),
            tf.keras.layers.Dense(600, activation='sigmoid'),
            tf.keras.layers.Dense(800, activation='sigmoid'),
            tf.keras.layers.Dense(1000, activation='sigmoid'),
            tf.keras.layers.Dense(800, activation='sigmoid'),
            tf.keras.layers.Dense(600, activation='sigmoid'),
            tf.keras.layers.Dense(self.procesador.dimY, activation='linear')
        ]),
        tf.keras.Sequential([
            tf.keras.layers.LSTM(400, input_shape=(self.procesador.dimX,2), return_sequences=True),
            tf.keras.layers.LSTM(300, return_sequences=True),
            tf.keras.layers.LSTM(200, return_sequences=True),
            tf.keras.layers.LSTM(100, return_sequences=True),
            tf.keras.layers.LSTM(50, return_sequences=True),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(self.procesador.dimY, activation='linear')
        ]),
        tf.keras.Sequential([
            tf.keras.layers.LSTM(300, input_shape=(self.procesador.dimX,2), kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), return_sequences=True),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.LSTM(300, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), return_sequences=True),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.LSTM(200, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), return_sequences=True),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.LSTM(100, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), return_sequences=True),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.LSTM(50, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), return_sequences=True),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(6000, activation='tanh', kernel_regularizer=l2(0.01)),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Dense(3600, activation='relu', kernel_regularizer=l2(0.01)),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Dense(2160, activation='tanh', kernel_regularizer=l2(0.01)),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Dense(1296, activation='relu', kernel_regularizer=l2(0.01)),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Dense(777, activation='tanh', kernel_regularizer=l2(0.01)),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Dense(466, activation='relu', kernel_regularizer=l2(0.01)),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Dense(167, activation='tanh', kernel_regularizer=l2(0.01)),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Dense(self.procesador.dimY, activation='linear')
        ]),
        tf.keras.Sequential([
            tf.keras.layers.LSTM(300, input_shape=(self.procesador.dimX,2), return_sequences=True),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.LSTM(300, return_sequences=True),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.LSTM(200, return_sequences=True),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.LSTM(100, return_sequences=True),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.LSTM(50, return_sequences=True),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Dense(self.procesador.dimY, activation='linear')
        ]),
            #tf.keras.layers.Dropout(0.3),
            #tf.keras.layers.LSTM(300, return_sequences=True),
            #tf.keras.layers.Dropout(0.3),
            #tf.keras.layers.LSTM(300, return_sequences=True),
            #tf.keras.layers.Dropout(0.3),
            #tf.keras.layers.LSTM(300, return_sequences=True),
            #tf.keras.layers.Dropout(0.3),
            #tf.keras.layers.LSTM(300, return_sequences=True),
            #tf.keras.layers.Dropout(0.3),
            #tf.keras.layers.LSTM(300, return_sequences=True),
            #tf.keras.layers.Dropout(0.3),
            #tf.keras.layers.LSTM(300, return_sequences=True),
            #tf.keras.layers.Dropout(0.3),
            #tf.keras.layers.LSTM(200, return_sequences=True),
            #tf.keras.layers.Dropout(0.3),
            #tf.keras.layers.LSTM(100, return_sequences=True),
            #tf.keras.layers.Dropout(0.3),
            #tf.keras.layers.Flatten(),
            #tf.keras.layers.Dropout(0.3),
    ]

  def loadPreset(self, indice):
    self.validarPresets()
    if tf.test.gpu_device_name():
      with tf.device('/GPU:0'):
        self.model = self.preset[indice]
        self.configModel()
    else:
      self.model = self.preset[indice]
      self.configModel()

  def validarPresets(self):
    if "preset" not in self.__dict__:
      self.makePresets()

  def configModel(self):
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
    self.model.compile(loss='mean_squared_error', optimizer='adam')
    self.model.summary()

  def loadModel(self, model):
    self.model = model
    self.configModel()

  def train(self, epochs = 500, batch_size = 20, testSize = 0.2):
    self.model.fit(
                self.procesador.X_train,
                self.procesador.Y_train[:, :, -1],
                epochs = epochs,
                batch_size = batch_size,
                validation_split = testSize,
                #validation_data = (self.procesador.X_test#[:, :, -1]
                #                   , self.procesador.Y_test[:, :, -1]),
                callbacks=[self.CustomCallback()]
              )
    loss = self.model.evaluate(self.procesador.X_test#[:, :, -1]
                               , self.procesador.Y_test[:, :, -1])
    print(f'Pérdida en datos de prueba: {loss}')

  def predict(self, data):
    return self.model.predict(data)

In [ ]:
# @title Objeto encargado de procesar las predicciones, graficas y mostrar comparaciones
class ProcesadorDeResultados:
  def __init__(self):
    self.contexto = None

  def procesar(self, X, Y, prediccion, escalado = 1000):
    self.escalado = escalado
    self.realResult = []
    self.predictedResult = []
    self.X = X
    self.Y = Y
    self.prediccion = prediccion
    for i in range(0,len(X)):

      ejeXPrediccion = Y[i,:,0]
      ejeYPrediccion = prediccion[i, :]

      datosPredichos = np.column_stack((ejeXPrediccion, ejeYPrediccion))

      svgReal = self.makeSvgData(X[i], Y[i], color = "green")
      svgPredicted = self.makeSvgData(X[i], datosPredichos, color = "red")

      color = self.makeRandomHexColor()
      self.realResult.append(
        {
          "data": svgReal,
          "color": color
        }
      )
      self.predictedResult.append(
          {
          "data": svgPredicted,
          "color": color
        }
      )

  def makeSvgData(self, ejeX, ejeY, color):
    ejeX = ejeX * self.escalado
    ejeY = ejeY * self.escalado
    nuevos_datos = np.concatenate((ejeX, ejeY), axis=0)
    svg = ""
    for i in nuevos_datos:
      svg = svg + "L"
      for j in i:
        svg = svg + f"{round(j, 2)},"
      svg = svg[:-1]
    svg = "M" + svg[1:]
    return svg

  def makeRandomHexColor(self):
    rojo = random.randint(0, 255)
    verde = random.randint(0, 255)
    azul = random.randint(0, 255)
    color_hex = "#{:02X}{:02X}{:02X}".format(rojo, verde, azul)
    return color_hex

  def showAllPredicted(self):
    self.graficarSvg(self.predictedResult)

  def graficarSvg(self, args):
    paths = ""
    for arg in args:
      data = arg["data"]
      color = arg["color"]
      paths = paths + f"""
        <path fill="none" stroke-width="2" stroke="{color}" stroke-dasharray="none" opacity="1" d="{data}">
      </path>
      """
    html_code = f"""
    <svg class="svg-datos" xmlns="http://www.w3.org/2000/svg">
      {paths}
    </svg>
    <script>
      svgs = document.getElementsByClassName("svg-datos");
          for (let svg of svgs){{
              path = svg.children[0];
              x = path.getBoundingClientRect().width;
              y = path.getBoundingClientRect().height;
              svg.style.width = x*1.5 + "px";
              svg.style.height = y*1.5 + "px";
          }}
    </script>
    """

    display(HTML(html_code))

  def showAllReal(self):
    color = self.makeRandomHexColor()
    self.graficarSvg(self.realResult)

  def showComparation(self):
    for indice in range(len(self.realResult)):
      predict = copy.deepcopy(self.predictedResult[indice])
      real = copy.deepcopy(self.realResult[indice])
      self.showContext(indice)
      predict["color"] = "red";
      real["color"] = "green"

      self.graficarSvg([predict, real])

  def showContext(self, indice):
    if not(self.contexto is None):
      contexto = self.contexto[indice]
      print("Nombre:",contexto["nombre"])
      print("Alias:",contexto["alias"])
      accion = contexto["accion"]
      print("Precio:",accion.price)
      print("Rendimiento completo:",accion.performance)
      print("Plazo:",accion.plazo)

      primerDato = accion.datos[0, -1]
      ultimoDato = accion.datos[-1, -1]
      rendimiento = accion.performance

      distanciaTotal = abs(primerDato - ultimoDato)


      ultimoDatoLimitadoReal = self.Y[indice, -1, -1] * self.escalado
      distanciaLimitadaReal = primerDato - ultimoDatoLimitadoReal
      rendimientoLimitadoReal =  (rendimiento - 1) / distanciaTotal * distanciaLimitadaReal + 1

      ultimoDatoLimitadoPredicho = self.prediccion[indice, -1] * self.escalado
      distanciaLimitadaPredicha = primerDato - ultimoDatoLimitadoPredicho
      rendimientoLimitadoPredicho =  (rendimiento - 1) / distanciaTotal * distanciaLimitadaPredicha + 1

      tamañoReal = accion.datos.shape[0]
      tamañoLimitado = self.X.shape[1] + self.Y.shape[1]
      print("Plazo de tiempo limitado", tamañoLimitado/tamañoReal)
      print(f"Rendimiento limitado real {rendimientoLimitadoReal}% - Precio {accion.price / accion.performance * rendimientoLimitadoReal}")
      print(f"Rendimiento limitado predicho {rendimientoLimitadoPredicho}% - Precio {accion.price / accion.performance * rendimientoLimitadoPredicho}")
      print(f"Margen de error {rendimientoLimitadoPredicho - rendimientoLimitadoReal}")

      #print(f"""
      #      primerDato {primerDato}
      #      ultimoDato {ultimoDato}
      #      ultimoDatoLimitadoReal {ultimoDatoLimitadoReal}
      #      ultimoDatoLimitadoPredicho {ultimoDatoLimitadoPredicho}
      #      distanciaTotal {distanciaTotal}
      #      distanciaLimitadaReal {distanciaLimitadaReal}
      #""")

  def setContexto(self, contexto):
    self.contexto = contexto

# Implementacion y uso de los objetos para trabajar

In [ ]:
contenedorAcciones = ContenedorAcciones()

In [ ]:
contenedorAcciones.buscarDatos(lista_acciones_bmv_capitales, show_progress=True)
contenedorAcciones.saveData("/content/drive/MyDrive/IA/acciones/acciones_datos_bmv_capitales.json")

In [ ]:
contenedorAcciones.loadData("/content/drive/MyDrive/IA/acciones/acciones_datos.json")

In [ ]:
datos = contenedorAcciones.getData()

In [ ]:
procesador = ProcesadorDatos(datos, limiteDatos=214)
procesador.procesar()

In [ ]:
procesador.X_train.shape, procesador.X_test.shape, procesador.Y_train.shape, procesador.Y_test.shape, procesador.dimX

In [ ]:
modelo = Modelo()
modelo.setProcesador(procesador)

In [ ]:
modelo.loadPreset(6)

In [ ]:
modelo.model = tf.keras.models.load_model("/content/drive/MyDrive/IA/acciones/modelo_80acciones_bSize12_arquitectura5_capas5.h5")

In [ ]:
modelo.train(2, batch_size = 100)

In [ ]:
modelo.model.save("/content/drive/MyDrive/IA/acciones/modelo_80acciones_bSize12_arquitectura5_capas5.h5")

In [ ]:
predicciones = modelo.predict(procesador.X_test)
predicciones.shape

In [ ]:
procesadorResultados = ProcesadorDeResultados()
procesadorResultados.procesar(procesador.X_test, procesador.Y_test, predicciones)
procesadorResultados.setContexto(procesador.testAcciones)
procesadorResultados.showAllReal()
procesadorResultados.showAllPredicted()
#procesadorResultados.showComparation()

In [ ]:
# @title Acciones de pruebas
lista_acciones_pruebas = [
    {
        "nombre": "PINFRA:BMV",
        "alias": "Promotora Y Operadora de Infraestructura, SAB DE CV"
    },
    {
        "nombre": "GFINBURO:BMV",
        "alias": "Grupo Financiero Inbursa"
    },
    {
        "nombre": "ESW00:CME_EMINIS",
        "alias": "Contrato continuo de E-mini S&P 500"
    },
    {
        "nombre": "GENTERA:BMV",
        "alias": "Gentera"
    },
    {
        "nombre": "TRAXIONA:BMV",
        "alias": "Grupo Traxion SAB de CV"
    },
    {
        "nombre": "AC:BMV",
        "alias": "Arca Continental SAB de CV"
    },
    {
        "nombre": "NQW00:CME_EMINIS",
        "alias": "Contrato continuo de E-mini NASDAQ 100"
    },
]

In [ ]:
contenedorAcciones_pruebas = ContenedorAcciones()


In [ ]:
contenedorAcciones_pruebas.buscarDatos(lista_acciones_pruebas, show_progress=True)
contenedorAcciones_pruebas.saveData("/content/drive/MyDrive/IA/acciones/acciones_datos_pruebas.json")

In [ ]:
contenedorAcciones_pruebas.loadData("/content/drive/MyDrive/IA/acciones/acciones_datos_pruebas.json")

In [ ]:
datos_pruebas = contenedorAcciones_pruebas.getData()

In [ ]:
procesador_pruebas = ProcesadorDatos(datos_pruebas, trainSize=0.2857, limiteDatos=215)
procesador_pruebas.procesar()

In [ ]:
procesador_pruebas.X_train.shape, procesador_pruebas.X_test.shape, procesador_pruebas.Y_train.shape, procesador_pruebas.Y_test.shape, procesador_pruebas.profundidad

In [ ]:
predicciones_pruebas = modelo.predict(procesador_pruebas.X_test)
predicciones_pruebas.shape

In [ ]:
procesadorResultados.procesar(procesador_pruebas.X_test, procesador_pruebas.Y_test, predicciones_pruebas)
procesadorResultados.showAllReal()
procesadorResultados.showAllPredicted()
procesadorResultados.setContexto(procesador_pruebas.testAcciones)
procesadorResultados.showComparation()

In [ ]:
cemex= []
for dato in datos:
  if(dato["alias"] == "Cemex"):
    cemex.append(dato
                 )

In [ ]:
for i in range(1):
  cemex.append(cemex[0])

In [ ]:
procesador_pruebas = ProcesadorDatos(cemex, trainSize=0.5, limiteDatos=215)
procesador_pruebas.procesar()

In [ ]:
procesador_pruebas.X_train.shape, procesador_pruebas.X_test.shape, procesador_pruebas.Y_train.shape, procesador_pruebas.Y_test.shape, procesador_pruebas.profundidad

In [ ]:
predicciones_pruebas = modelo.predict(procesador_pruebas.X_test)
predicciones_pruebas.shape

In [ ]:
procesadorResultados = ProcesadorDeResultados()
procesadorResultados.procesar(procesador_pruebas.X_test, procesador_pruebas.Y_test, predicciones_pruebas)
procesadorResultados.setContexto(procesador_pruebas.testAcciones)
procesadorResultados.showComparation()